# 5. yfinance를 이용한 주식 데이터 수집

### 5-1. 필수 라이브러리 Import


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import warnings
import numpy as np
warnings.filterwarnings('ignore')


### 5-2. RSI 계산 함수

In [ ]:
def calculate_rsi(prices, window=14):
    
    # 전일 대비 가격 변화량 계산
    delta = prices.diff()
    
    # 상승일의 가격 상승폭만 추출 (하락일은 0으로 처리)
    # where 조건: delta > 0인 경우만 해당 값 사용, 나머지는 0
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    
    # 하락일의 가격 하락폭만 추출 (상승일은 0으로 처리)
    # -delta로 하락폭을 양수로 변환
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    
    # RS (Relative Strength) = 평균 상승폭 / 평균 하락폭
    rs = gain / loss
    
    # RSI = 100 - (100 / (1 + RS))
    # RSI가 70 이상이면 과매수, 30 이하면 과매도 구간으로 해석
    rsi = 100 - (100 / (1 + rs))
    
    return rsi


### 5-3. 시간 조정 함수

In [ ]:
def adjust_time_to_hour(df):
    
    # Datetime 컬럼이 존재하는지 확인
    if 'Datetime' in df.columns:
        # 문자열을 pandas datetime 객체로 변환
        df['Datetime'] = pd.to_datetime(df['Datetime'])
        
        # 시간을 시간 단위로 내림 (분, 초를 0으로 만듦)
        # floor('H'): 시간 단위로 내림 처리
        # 예: 2024-01-15 13:30:45 → 2024-01-15 13:00:00
        df['Datetime'] = df['Datetime'].dt.floor('H')
        
        # 동일한 시간이 여러 개 있는 경우 마지막 값만 유지
        # keep='last': 중복된 값 중 마지막 행을 유지
        df = df.drop_duplicates(subset=['Datetime'], keep='last')
        
    return df


### 5-4. 기술적 지표 추가 함수 (1시간 간격용)

In [ ]:
def add_technical_features(df):
    
    # === 수익률 계산 ===
    # 전 시간 대비 수익률 계산 (Close[t] - Close[t-1]) / Close[t-1]
    df['Returns'] = df['Close'].pct_change()
    
    # 로그 수익률 계산 ln(Close[t] / Close[t-1])
    # 연속복리 개념으로, 작은 변화에서는 일반 수익률과 유사
    df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
    
    # === 이동평균 (Simple Moving Average) ===
    # 단순 이동평균: 지정 기간의 평균 가격
    df['SMA_10'] = df['Close'].rolling(window=10).mean()  # 10시간 평균
    df['SMA_20'] = df['Close'].rolling(window=20).mean()  # 20시간 평균
    df['SMA_50'] = df['Close'].rolling(window=50).mean()  # 50시간 평균
    
    # === 지수이동평균 (Exponential Moving Average) ===
    # 최근 데이터에 더 높은 가중치를 부여하는 이동평균
    df['EMA_12'] = df['Close'].ewm(span=12).mean()  # 12시간 EMA
    df['EMA_26'] = df['Close'].ewm(span=26).mean()  # 26시간 EMA
    
    # === MACD (Moving Average Convergence Divergence) ===
    # 단기 EMA와 장기 EMA의 차이로 추세 변화를 파악
    df['MACD'] = df['EMA_12'] - df['EMA_26']  # MACD 라인
    df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()  # 시그널 라인 (9시간 EMA)
    df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']  # 히스토그램 (매수/매도 신호)
    
    # === RSI (Relative Strength Index) ===
    # 과매수/과매도 상태를 나타내는 모멘텀 지표 (0~100)
    df['RSI'] = calculate_rsi(df['Close'])
    
    # === 볼린저 밴드 (Bollinger Bands) ===
    # 가격의 변동성을 기반으로 한 기술적 지표
    df['BB_Middle'] = df['Close'].rolling(window=20).mean()  # 중심선 (20시간 이동평균)
    bb_std = df['Close'].rolling(window=20).std()  # 20시간 표준편차
    df['BB_Upper'] = df['BB_Middle'] + (bb_std * 2)  # 상단 밴드 (평균 + 2*표준편차)
    df['BB_Lower'] = df['BB_Middle'] - (bb_std * 2)  # 하단 밴드 (평균 - 2*표준편차)
    df['BB_Width'] = df['BB_Upper'] - df['BB_Lower']  # 밴드 폭 (변동성 지표)
    # 현재 가격이 밴드 내에서 어느 위치에 있는지 (0~1, 0.5가 중앙)
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    
    # === 변동성 지표 ===
    # 수익률의 표준편차로 가격 변동성 측정
    df['Volatility_10'] = df['Returns'].rolling(window=10).std()  # 10시간 변동성
    df['Volatility_20'] = df['Returns'].rolling(window=20).std()  # 20시간 변동성
    
    # === 가격 변화 지표 ===
    # 시가 대비 종가 변화 (절대값)
    df['Price_Change'] = df['Close'] - df['Open']
    
    # 시가 대비 종가 변화 (백분율)
    df['Price_Change_Pct'] = (df['Close'] - df['Open']) / df['Open'] * 100
    
    # === High-Low 스프레드 ===
    # 당일 최고가와 최저가의 차이 (일중 변동폭)
    df['HL_Spread'] = df['High'] - df['Low']
    
    # 종가 대비 일중 변동폭 비율
    df['HL_Spread_Pct'] = (df['High'] - df['Low']) / df['Close'] * 100
    
    # === 시간 특성 ===
    # 시간대별 패턴 분석을 위한 특성
    df['Hour'] = df['Datetime'].dt.hour  # 시간 (0~23)
    df['DayOfWeek'] = df['Datetime'].dt.dayofweek  # 요일 (0=월요일, 6=일요일)
    df['Month'] = df['Datetime'].dt.month  # 월 (1~12)
    df['Quarter'] = df['Datetime'].dt.quarter  # 분기 (1~4)
    
    # === 거래시간 분류 ===
    # 미국 주식시장 기준 거래시간 분류
    df['Is_Trading_Hours'] = ((df['Hour'] >= 9) & (df['Hour'] <= 16)).astype(int)  # 정규 거래시간
    df['Is_Market_Open'] = ((df['Hour'] >= 9) & (df['Hour'] < 16)).astype(int)     # 시장 개장시간
    df['Is_Premarket'] = ((df['Hour'] >= 4) & (df['Hour'] < 9)).astype(int)       # 프리마켓 (4:00-9:30)
    df['Is_Aftermarket'] = ((df['Hour'] >= 16) & (df['Hour'] <= 20)).astype(int)  # 애프터마켓 (16:00-20:00)
    df['Is_Extended_Hours'] = (df['Is_Premarket'] | df['Is_Aftermarket']).astype(int)  # 연장거래시간
    
    return df


### 5-5. 1시간 간격 주식 데이터 수집 함수

In [ ]:
def get_hourly_stock_data(ticker, days=365, save_to_csv=True):
    
    try:
        # yfinance 1시간 간격 데이터 제약사항 확인 (최대 730일)
        if days > 730:
            days = 730
        
        # 현재 날짜 기준으로 시작일과 종료일 계산
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        
        # yfinance를 사용하여 주식 데이터 다운로드
        stock_data = yf.download(
            ticker,  # 티커 심볼
            start=start_date.strftime('%Y-%m-%d'),  # 시작일 (YYYY-MM-DD 형식)
            end=end_date.strftime('%Y-%m-%d'),      # 종료일 (YYYY-MM-DD 형식)
            interval='1h',    # 1시간 간격
            prepost=True,     # 시장 외 시간 데이터 포함 (프리마켓, 애프터마켓)
            progress=False    # 진행상황 표시 안 함
        )
        
        # 데이터가 비어있는 경우 None 반환
        if stock_data.empty:
            return None
        
        # 인덱스(날짜/시간)를 일반 컬럼으로 변환
        stock_data = stock_data.reset_index()
        
        # === 컬럼명 정리 작업 ===
        # yfinance의 다양한 반환 형식에 대응
        
        # 'Date' 컬럼이 있으면 'Datetime'으로 변경
        if 'Date' in stock_data.columns:
            stock_data = stock_data.rename(columns={'Date': 'Datetime'})
        # 첫 번째 컬럼이 시간 데이터인 경우 'Datetime'으로 변경
        elif stock_data.columns[0] not in ['Datetime', 'Date']:
            stock_data = stock_data.rename(columns={stock_data.columns[0]: 'Datetime'})
        
        # === 멀티레벨 컬럼 처리 ===
        # yfinance가 때로 계층적 컬럼 구조로 데이터를 반환하는 경우
        if isinstance(stock_data.columns, pd.MultiIndex):
            new_columns = []
            for col in stock_data.columns:
                if isinstance(col, tuple):
                    # 튜플의 첫 번째 요소가 시간 관련이면 'Datetime'으로
                    if col[0] == 'Datetime' or 'Date' in str(col[0]):
                        new_columns.append('Datetime')
                    else:
                        # 그 외는 첫 번째 요소만 사용 (Open, High, Low, Close, Volume)
                        new_columns.append(col[0])
                else:
                    new_columns.append(col)
            stock_data.columns = new_columns
        
        # 최종적으로 'Datetime' 컬럼이 없으면 첫 번째 컬럼을 사용
        if 'Datetime' not in stock_data.columns:
            stock_data = stock_data.rename(columns={stock_data.columns[0]: 'Datetime'})
        
        # === 데이터 전처리 및 특성 추가 ===
        # 시간을 정시로 조정 (예: 13:30 → 13:00)
        stock_data = adjust_time_to_hour(stock_data)
        
        # 기술적 지표 계산 및 추가
        stock_data = add_technical_features(stock_data)
        
        # === CSV 파일 저장 ===
        if save_to_csv:
            # 파일명 형식: {TICKER}_1hour_data_{DAYS}days.csv
            filename = f"{ticker}_1hour_data_{days}days.csv"
            stock_data.to_csv(filename, index=False)  # 인덱스 제외하고 저장
        
        return stock_data
        
    except Exception as e:
        # 에러 발생 시 None 반환
        return None


### 5-6. 다중 티커 1시간 데이터 수집 함수

In [ ]:
def get_multiple_tickers_hourly(tickers, days=365, save_individual=True, save_combined=True):
    
    # 수집된 데이터를 저장할 딕셔너리 초기화
    all_data = {}
    
    # 각 티커에 대해 순차적으로 데이터 수집
    for ticker in tickers:
        # 개별 티커 데이터 수집 (save_to_csv는 save_individual 설정에 따라)
        data = get_hourly_stock_data(ticker, days=days, save_to_csv=save_individual)
        
        # 수집된 데이터가 유효한 경우 딕셔너리에 저장
        if data is not None:
            all_data[ticker] = data
    
    # === 통합 데이터 파일 생성 ===
    if save_combined and all_data:
        # 모든 티커 데이터를 하나로 합칠 빈 데이터프레임 생성
        combined_data = pd.DataFrame()
        
        # 각 티커 데이터에 티커 컬럼 추가 후 통합
        for ticker, data in all_data.items():
            # 원본 데이터를 복사하여 수정
            ticker_data = data.copy()
            
            # 티커 식별을 위한 'Ticker' 컬럼 추가
            ticker_data['Ticker'] = ticker
            
            # 기존 통합 데이터에 현재 티커 데이터 추가
            # ignore_index=True: 인덱스를 새로 생성 (연속적인 번호)
            combined_data = pd.concat([combined_data, ticker_data], ignore_index=True)
        
        # 통합 데이터 CSV 파일 저장
        combined_filename = f"multiple_stocks_1hour_data_{days}days.csv"
        combined_data.to_csv(combined_filename, index=False)
    
    # 티커별 데이터가 담긴 딕셔너리 반환
    return all_data


### 5-7. 데이터 요약 분석 함수

In [ ]:
def analyze_data_summary(data_dict):
    
    # 딕셔너리의 각 티커와 데이터에 대해 반복
    for ticker, data in data_dict.items():
        # 데이터가 유효한 경우에만 분석 진행
        if data is not None:
            # 전체 데이터에서 결측치(NaN) 개수 계산
            # isnull(): 각 셀이 결측치인지 True/False 반환
            # sum().sum(): 먼저 각 컬럼별 결측치 개수를 구한 후, 전체 합계 계산
            missing_count = data.isnull().sum().sum()
            
            # 결과 출력 (데이터 포인트 개수와 결측치 개수)
            # len(data): 총 행(데이터 포인트) 개수
            # :,: 천 단위 구분자 추가 (예: 1000 → 1,000)
            print(f"{ticker}: {len(data):,}개 포인트, 결측치: {missing_count}개")


### 5-8. 주요 종목 1시간 데이터 수집 실행


In [ ]:
# === 수집 대상 주요 종목 설정 ===
# 대형주 기술주 중심으로 선정 (시가총액, 거래량, 변동성 고려)
# - AAPL: 애플 (아이폰, 맥북 등 하드웨어)
# - AMZN: 아마존 (전자상거래, 클라우드 서비스)
# - TSLA: 테슬라 (전기자동차, 에너지)
# - GOOGL: 구글 (검색엔진, 광고, 클라우드)
# - MSFT: 마이크로소프트 (윈도우, 오피스, 클라우드)
tickers = ['AAPL', 'AMZN', 'TSLA', 'GOOGL', 'MSFT']

# === 1시간 간격 데이터 수집 실행 ===
# days=365: 1년간의 데이터 수집 (충분한 학습 데이터 확보)
# save_individual=True: 각 종목별 개별 CSV 파일 생성
# save_combined=True: 모든 종목을 통합한 CSV 파일도 생성
all_stock_data = get_multiple_tickers_hourly(tickers, days=365)

# === 수집 결과 요약 분석 ===
# 각 종목별 데이터 포인트 개수와 결측치 개수 확인
analyze_data_summary(all_stock_data)

# 수집 완료된 종목 개수 출력
print(f"\\n데이터 수집 완료: {len(all_stock_data)}개 종목")


### 5-9. AAPL 데이터 상세 분석

In [ ]:
# === AAPL 데이터 존재 여부 확인 및 상세 분석 ===
if 'AAPL' in all_stock_data:
    # AAPL 1시간 데이터를 변수에 저장
    aapl_1h = all_stock_data['AAPL']
    
    # === 기본 데이터 미리보기 ===
    print("AAPL 1시간 데이터 미리보기:")
    # 핵심 OHLCV 컬럼만 선택하여 상위 5개 행 출력
    # OHLCV: Open(시가), High(고가), Low(저가), Close(종가), Volume(거래량)
    print(aapl_1h[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']].head())
    
    # === 거래시간 데이터 필터링 ===
    # LSTM 학습에는 정규 거래시간 데이터가 더 안정적
    # Is_Trading_Hours == 1: 미국 시장 정규 거래시간 (9:30-16:00 ET)
    trading_hours = aapl_1h[aapl_1h['Is_Trading_Hours'] == 1]
    
    # === 학습용 데이터 분석 ===
    print(f"\\n학습용 데이터 분석:")
    print(f"전체 시간 개수: {len(aapl_1h):,}개")  # 프리마켓, 애프터마켓 포함
    print(f"거래시간 개수: {len(trading_hours):,}개")  # 정규 거래시간만
    
    # === 기술적 지표 컬럼 확인 ===
    # LSTM 모델의 입력 특성으로 사용할 주요 기술적 지표들
    tech_indicators = [col for col in aapl_1h.columns if col in ['SMA_10', 'SMA_20', 'RSI', 'MACD', 'BB_Upper', 'BB_Lower']]
    print(f"기술적 지표 컬럼: {tech_indicators}")
    
    # === 데이터 품질 분석 ===
    # 결측치가 있는 기술적 지표 확인
    missing_indicators = aapl_1h[tech_indicators].isnull().sum()
    print(f"\\n기술적 지표별 결측치:")
    for indicator, missing_count in missing_indicators.items():
        if missing_count > 0:
            print(f"  {indicator}: {missing_count}개")
    
    # === 데이터 범위 정보 ===
    print(f"\\n데이터 기간:")
    print(f"시작: {aapl_1h['Datetime'].min()}")
    print(f"종료: {aapl_1h['Datetime'].max()}")
    print(f"총 기간: {(aapl_1h['Datetime'].max() - aapl_1h['Datetime'].min()).days}일")


### 5-10. 데이터 수집 완료 요약

In [ ]:
# === 데이터 수집 완료 최종 요약 ===
print("\\n 주식 데이터 수집 완료!")

# 성공적으로 수집된 종목 목록 출력
print(f"수집된 종목: {list(all_stock_data.keys())}")

# === 생성된 CSV 파일 목록 확인 ===
print("\\n 생성된 CSV 파일:")

# 현재 디렉토리의 파일 목록을 가져와서 CSV 파일만 필터링
import os
# 리스트 컴프리헨션으로 CSV 파일 중 수집 대상 티커가 포함된 파일만 선택
csv_files = [f for f in os.listdir('.') if f.endswith('.csv') and any(ticker in f for ticker in tickers)]

# 각 생성된 CSV 파일을 출력
for file in csv_files:
    print(f"  {file}")